In [50]:
!pip install pygraphviz

<img src="./imgs/Logo-unifi.png" align="left" width="15%" style="float:left; display: inline-block">

<div align="right" style="font-size:12px">
    Abdullah Chaudhry <br>
Claudia Raffaelli
        </div>


<br><br>
<div align="center"><b> Abstract </b><br>
These slides describe a technique to calculate the Betweenness Centrality of a graph's nodes using Bellman-Ford's algorithm</div>

In [51]:
__AUTHORS__ = {'ac': ("Abdullah Chaudhry", 
                      "abdullah.chaudhry@stud.unifi.it",
                      "https://github.com/chabdullah"),
               'cr': ("Claudia Raffaelli", 
                      "claudia.raffaelli@stud.unifi.it", 
                      "https://github.com/ClaudiaRaffaelli",)}

__KEYWORDS__ = ['Python', 'Jupyter', 'notebooks', 'Bellman-Ford', 'Betweenness-Centrality', 'graphs']

<div align="center"><img src="./imgs/Python-logo-notext.svg"></div>

## Introduction
During the next slides we will see how to:
* Handle large graphs
* Implement two variations of the Bellman-Ford algorithm
* Implement the Betweenness Centrality algorithm

We start by loading the project:

In [52]:
from graph_manager import GraphManager
import networkx as nx
import json
import math
import random
import time
from collections import deque
import matplotlib as plt

In [53]:
with open("./dati-json/dpc-covid19-ita-province.json") as f:
    parsed_file = json.load(f)

# Building the graph of provinces
provinces_already_annotated = []
P = GraphManager()
for province_data in parsed_file:
    # extracting information from the JSON
    if province_data['sigla_provincia'] != '' and province_data[
        'sigla_provincia'] not in provinces_already_annotated:
        provinces_already_annotated.append(province_data['sigla_provincia'])
        province = province_data['denominazione_provincia']
        position_x = province_data['long']
        position_y = province_data['lat']
        # adding each province to the graph
        P.add_node_to_graph(province, position_x, position_y)

# Building the graph of doubles
R = GraphManager()
# Generate 2000 pairs of double (x,y)
for i in range(2000):
    x = random.randrange(30, 50)
    y = random.randrange(10, 20)
    R.add_node_to_graph(str(i), x, y)

## Adding edges

While handling large graphs, heavy operations like adding edges to the graph itself can be challenging. 
Here, we see our implementatin of this task. 

We keep in mind that two nodes <code>a</code> and <code>b</code> are connected by an edge if the following holds: 
* if <code>x,y</code> is the position of <code>a</code>, then <code>b</code> is in position <code>z,w</code> with <code>z</code> in <code>[x-d,x+d]</code> and <code>w</code> in <code>[y-d, y+d]</code>, with <code>d=0.8</code>.

Here we see the main steps:
* Is created a list of pairs with this structure: <code>(node_name, (x, y))</code>,
* This list is sorted by the <code>x</code> value,
* We iterate over the sorted list with an index <code>i</code>,
* We also keep an index <code>j</code> that can help us compare the node <code>i</code> to this other node,
* If the nodes indexed by <code>i</code> and <code>j</code> are close enough relatively to <code>x</code> and <code>y</code>, then we add an edge, with weight the Eucledian distance between the two,
* If the node <code>i</code> and <code>j</code> are close only with respect to <code>x</code>, we must check the node <code>i</code> with <code>j+1</code>, since the list is only ordered wrt <code>x</code>, and not <code>y</code>,
* If the node indexed by <code>i</code> and the node indexed by <code>j</code> are not close enough relatively to <code>x</code>, of course <code>i</code> and <code>j+1</code> will also not be close wrt <code>x</code> because of the sorting. In this case we increment <code>i</code> and set <code>j=i+1</code>. 

In [54]:
# Inserting the edges according to the distance between each node
%timeit P.add_edges()
%timeit R.add_edges()

100 loops, best of 5: 2.05 ms per loop
10 loops, best of 5: 143 ms per loop


# Shortest paths

In graph theory, the **shortest path problem** is the problem of finding a path between two vertices (or nodes) in a graph such that the sum of the weights of its constituent edges is minimized.
![Shortest path](imgs/shortest_path.png)

Applications:
- Shortest path algorithms are applied to automatically find directions between physical locations, such as driving directions on web mapping websites like Google Maps
- Considering a nondeterministic abstract machine as a graph where vertices describe states and edges describe possible transitions, shortest path algorithms can be used to find an optimal sequence of choices to reach a certain goal state

# Bellman-Ford algorithm

The Bellman–Ford algorithm is an algorithm that computes shortest paths from a single source vertex to all of the other vertices in a **weighted digraph**. It is slower than Dijkstra's algorithm for the same problem, but more versatile, as it is capable of handling graphs in which some of the edge weights are negative numbers. 


Negative edge weights are found in various applications of graphs, hence the usefulness of this algorithm. If a graph contains a **"negative cycle"** (i.e. a cycle whose edges sum to a negative value) that is reachable from the source, then there is no cheapest path: any path that has a point on the negative cycle can be made cheaper by one more walk around the negative cycle. In such a case, the Bellman–Ford algorithm can detect and report the negative cycle.

## First intuition
Like Dijkstra's algorithm, Bellman–Ford proceeds by relaxation, in which approximations to the correct distance are replaced by better ones until they eventually reach the solution. In both algorithms, the approximate distance to each vertex is always an overestimate of the true distance, and is replaced by the minimum of its old value and the length of a newly found path.

Bellman–Ford algorithm simply relaxes all the edges, $|V|-1$ times, where $|V|$ is the number of vertices in the graph. In each of these repetitions, the number of vertices with correctly calculated distances grows, from which it follows that eventually all vertices will have their correct distances. 


## Toy Example

<table><tr>
<td> <img src="./imgs/bellman_0.png" alt="Iteration_0" style="width: 350px;"/> </td>
<td> <img src="./imgs/bellman_1.png" alt="Iteration_1" style="width: 350px;"/> </td>
<td> <img src="./imgs/bellman_2.png" alt="Iteration_2" style="width: 350px;"/> </td>
</tr></table>

- **Iteration 0:** all distances are initialized from source vertex A,
- **Iteration 1:** all edges are processed in the order (B,E), (D,B), (B,D), (A,B), (A,C), (D,C), (B,C), (E,D). Found all shortest paths which are at most 1 edge long,
- **Iteration 2:** processing again all edges. All shortest paths which are at most 2 edges long are found,
- **Iteration 3 and 4:** useless.

## Our First Implementation

```python
def bellman_ford(self, source_vertex):
	vertices = list(self.graph.nodes())
	
    distances = dict.fromkeys(self.graph.nodes(), math.inf)
	predecessors = dict.fromkeys(self.graph.nodes(), None)
	distances[source_vertex] = 0
    # relax edges
	count = len(vertices) - 1
	while count > 0:
		something_has_changed = False
		for (u, v) in self.graph.edges():
            # considering both the symmetric edges in the form (u, v) and (v, u)
			if distances[u] + float(self.graph[u][v]['label']) < distances[v]:
				distances[v] = distances[u] + float(self.graph[u][v]['label'])
				predecessors[v] = u
				something_has_changed = True
			if distances[v] + float(self.graph[v][u]['label']) < distances[u]:
				distances[u] = distances[v] + float(self.graph[v][u]['label'])
				predecessors[u] = v
				something_has_changed = True		
		if something_has_changed is False:
			break
		count -= 1
	return distances, predecessors
```

### Computation time
Bellman–Ford runs in $O(|V|\cdot |E|)$ time, where $|V|$ and $|E|$ are the number of vertices and edges respectively at its worst case. 

The Bellman–Ford algorithm may be improved in practice (although not in the worst case), as we did, by the observation that, if an iteration of the main loop of the algorithm terminates without making any changes, the algorithm can be immediately terminated, as subsequent iterations will not make any more changes. With this early termination condition, the main loop may in some cases use many fewer than $|V| − 1$ iterations, even though the worst case of the algorithm remains unchanged.

## New strategy
The **Shortest Path Faster Algorithm (SPFA)** is an improvement of the Bellman–Ford algorithm which computes single-source shortest paths in a weighted directed graph. The algorithm is believed to work well on random sparse graphs and is particularly suitable for graphs that contain negative-weight edges. However, the worst-case complexity of SPFA is the same as that of Bellman–Ford, so for graphs with nonnegative edge weights Dijkstra's algorithm is still preferred.

The basic idea of SPFA is the same as Bellman–Ford algorithm in that each vertex is used as a candidate to relax its adjacent vertices. The improvement over the latter is that instead of trying all vertices blindly, SPFA maintains a **queue** of candidate vertices and adds a vertex to the queue only if that vertex is relaxed. This process repeats until no more vertex can be relaxed.

## Our Second Implementation
```python
def bellman_ford_SPFA(self, source_vertex):	
	distances = dict.fromkeys(self.graph.nodes(), math.inf)
	already_in_queue = dict.fromkeys(self.graph.nodes(), False)
	predecessors = dict.fromkeys(self.graph.nodes(), math.inf)
	distances[source_vertex] = 0
    
	q = deque()
	q.append(source_vertex)
	already_in_queue[source_vertex] = True
	while len(q) > 0:
		u = q.popleft()
		for (u, v) in self.graph.edges(u):
			if distances[u] + float(self.graph[u][v]['label']) < distances[v]:
				distances[v] = distances[u] + float(self.graph[u][v]['label'])
				if not already_in_queue[v]:
					q.append(v)
					already_in_queue[v] = True
				predecessors[v] = u
			if distances[v] + float(self.graph[v][u]['label']) < distances[u]:
				distances[u] = distances[v] + float(self.graph[v][u]['label'])
				if not already_in_queue[u]:
					q.append(u)
					already_in_queue[u] = True
				predecessors[u] = v
	return distances, predecessors
```

### Computation time

The worst-case running time of the algorithm is $O(|V|\cdot|E|)$ , just like the standard Bellman-Ford algorithm. However experiments suggest that the average running time is $O(|E|)$. 

We can now compare the two strategies.

In [55]:
%timeit P.bellman_ford("Firenze")
%timeit P.bellman_ford_SPFA("Firenze")

100 loops, best of 5: 5.63 ms per loop
100 loops, best of 5: 2.15 ms per loop


In [56]:
distances, predecessors = P.bellman_ford_SPFA("Firenze")

In [57]:
distances

{'Agrigento': inf,
 'Alessandria': 3.3526890000000003,
 'Ancona': 2.507541,
 'Aosta': 4.675934,
 'Arezzo': 0.695748,
 'Ascoli Piceno': 3.0071630000000003,
 'Asti': 3.76418,
 'Avellino': 5.061338000000001,
 'Bari': 7.048739000000001,
 'Barletta-Andria-Trani': 6.467652000000001,
 'Belluno': 2.6032770000000003,
 'Benevento': 4.848592000000001,
 'Bergamo': 2.6031910000000003,
 'Biella': 3.922346,
 'Bologna': 0.730198,
 'Bolzano': 2.907397,
 'Brescia': 2.196202,
 'Brindisi': 8.49972,
 'Cagliari': inf,
 'Caltanissetta': inf,
 'Campobasso': 4.403401000000001,
 'Caserta': 4.7596870000000004,
 'Catania': inf,
 'Catanzaro': inf,
 'Chieti': 3.470136,
 'Como': 3.1084870000000002,
 'Cosenza': inf,
 'Cremona': 1.9388450000000002,
 'Crotone': inf,
 'Cuneo': 4.590156,
 'Enna': inf,
 'Fermo': 2.9489639999999997,
 'Ferrara': 1.169991,
 'Firenze': 0,
 'Foggia': 5.679577000000001,
 'Forlì-Cesena': 0.906381,
 'Frosinone': 3.0715460000000006,
 'Genova': 3.188689,
 'Gorizia': 3.706014,
 'Grosseto': 1.056529,

In [58]:
predecessors

{'Agrigento': inf,
 'Alessandria': 'Pavia',
 'Ancona': 'Pesaro e Urbino',
 'Aosta': 'Biella',
 'Arezzo': 'Firenze',
 'Ascoli Piceno': 'Rieti',
 'Asti': 'Alessandria',
 'Avellino': 'Campobasso',
 'Bari': 'Barletta-Andria-Trani',
 'Barletta-Andria-Trani': 'Foggia',
 'Belluno': 'Padova',
 'Benevento': 'Campobasso',
 'Bergamo': 'Cremona',
 'Biella': 'Novara',
 'Bologna': 'Firenze',
 'Bolzano': 'Trento',
 'Brescia': 'Mantova',
 'Brindisi': 'Taranto',
 'Cagliari': inf,
 'Caltanissetta': inf,
 'Campobasso': 'Chieti',
 'Caserta': 'Isernia',
 'Catania': inf,
 'Catanzaro': inf,
 'Chieti': "L'Aquila",
 'Como': 'Monza e della Brianza',
 'Cosenza': inf,
 'Cremona': "Reggio nell'Emilia",
 'Crotone': inf,
 'Cuneo': 'Asti',
 'Enna': inf,
 'Fermo': 'Macerata',
 'Ferrara': 'Bologna',
 'Firenze': inf,
 'Foggia': 'Benevento',
 'Forlì-Cesena': 'Firenze',
 'Frosinone': 'Rieti',
 'Genova': 'Piacenza',
 'Gorizia': 'Udine',
 'Grosseto': 'Siena',
 'Imperia': 'Savona',
 'Isernia': 'Chieti',
 "L'Aquila": 'Terni',

## Putting all together
```python
def bellman_ford_shortest_path(self, source_vertex, SPFA=True):
	if SPFA:
		distances, predecessors = self.bellman_ford_SPFA(source_vertex)
	else:
		distances, predecessors = self.bellman_ford(source_vertex)
	all__shortest_paths = []
	nodes = list(self.graph.nodes(data=True))
	for target_vertex in nodes:
		target_vertex = target_vertex[0]
		if predecessors[target_vertex] == math.inf:
			continue
		# using the predecessors of each node to build the shortest path
		shortest_path = []
		current_node = target_vertex
		shortest_path.append(target_vertex)
		while current_node != source_vertex:
			current_node = predecessors[current_node]
			# no path between the two nodes: exiting from the loop
			if current_node is None:
				break
			shortest_path.append(current_node)
		if len(shortest_path) != 1:
			all__shortest_paths.append(shortest_path)
	return all__shortest_paths
```

The output obtained by this computation:

In [59]:
P.bellman_ford_shortest_path("Firenze", SPFA=True)[:5]

[['Chieti', "L'Aquila", 'Terni', 'Perugia', 'Arezzo', 'Firenze'],
 ["L'Aquila", 'Terni', 'Perugia', 'Arezzo', 'Firenze'],
 ['Pescara', 'Chieti', "L'Aquila", 'Terni', 'Perugia', 'Arezzo', 'Firenze'],
 ['Teramo', "L'Aquila", 'Terni', 'Perugia', 'Arezzo', 'Firenze'],
 ['Matera',
  'Barletta-Andria-Trani',
  'Foggia',
  'Benevento',
  'Campobasso',
  'Chieti',
  "L'Aquila",
  'Terni',
  'Perugia',
  'Arezzo',
  'Firenze']]

In [60]:
%timeit P.bellman_ford_shortest_path("Firenze", SPFA=True)

100 loops, best of 5: 2.21 ms per loop



# Centrality 

In network analysis, indicators of **centrality** identify the most important vertices within a graph.
![title](./imgs/important_node.png)
Applications:
- Identifying the most influential person(s) in a social network
  
- Key infrastructure nodes in the Internet
  
- Super-spreaders of disease.

# Betweenness Centrality

Betweenness centrality, in Graph Theory, is a measure of centrality in a graph based on shortest paths. Vertices with high betweenness may have considerable influence within a network by virtue of their control over information passing between others. They are also the ones whose removal from the network will most disrupt communications between other vertices because they lie on the largest number of paths taken by messages.

More compactly the betweenness can be represented as:

**$g(v) =\sum\limits_{s\neq v\neq t \in V} \frac{\sigma_{st}(v)}{\sigma_{st}}$**


where **$\sigma_{st}$** is total number of shortest paths from node $s$ to node $t$ and **$\sigma_{st}(v)$** is the number of those paths that pass through $v$.

# Toy example

<img src="./imgs/bc_toy_example.png" width=600>

# Betweenness centrality in large graphs
The betweenness centrality of a node scales with the number of pairs of nodes. Therefore, the calculation may be rescaled by dividing through by the number of pairs of nodes not including $v$, so that $g(v)\in[0,1]$. The division is done by $(N−1)(N−2)$ for directed graphs and $\frac{(N−1)(N−2)}{2}$ for undirected graphs, where $N$ is the number of nodes.

### A handy benefit to betwenness centrality 
You don't need a fully connected graph or component to calculate it
![title](./imgs/component.png)

In [61]:
P.betweenness_centrality()

{'Agrigento': 0.0,
 'Alessandria': 0.030907457322551662,
 'Ancona': 0.0,
 'Aosta': 0.0,
 'Arezzo': 0.11410601976639713,
 'Ascoli Piceno': 0.14699011680143756,
 'Asti': 0.015813117699910152,
 'Avellino': 0.013297394429469901,
 'Bari': 0.0,
 'Barletta-Andria-Trani': 0.08733153638814016,
 'Belluno': 0.0,
 'Benevento': 0.11069182389937107,
 'Bergamo': 0.028032345013477088,
 'Biella': 0.014555256064690027,
 'Bologna': 0.1417789757412399,
 'Bolzano': 0.0,
 'Brescia': 0.0637915543575921,
 'Brindisi': 0.015453728661275831,
 'Cagliari': 0.0,
 'Caltanissetta': 0.0035938903863432167,
 'Campobasso': 0.14429469901168016,
 'Caserta': 0.00017969451931716083,
 'Catania': 0.00646900269541779,
 'Catanzaro': 0.001976639712488769,
 'Chieti': 0.19748427672955976,
 'Como': 0.007367475292003594,
 'Cosenza': 0.0,
 'Cremona': 0.0697214734950584,
 'Crotone': 0.0,
 'Cuneo': 0.00035938903863432165,
 'Enna': 0.0,
 'Fermo': 0.011680143755615454,
 'Ferrara': 0.027672955974842768,
 'Firenze': 0.07367475292003593,
 'F

## Some common algorithms

Calculating the betweenness centrality of all the vertices in a graph involves calculating the shortest paths between all pairs of vertices on a graph, which takes:
* $O(|V||E|)$ for unweighted graphs *([Brandes' algorithm](https://web.archive.org/web/20171013152036/http://algo.uni-konstanz.de/publications/b-fabc-01.pdf))*.
* $Θ(|V|^3)$ for weighted graphs *([Floyd–Warshall algorithm](https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm))

A single execution of the algorithms will find the shortest paths between all pairs of vertices. 

# Our implementation

Here is a snippet of our implementation, we used *Bellman-Ford's alghoritm* to find the shortest paths from a source node $i$ to $v$, with $v \in V$
```python
def betweenness_centrality(self):
    nodes = list(self.graph.nodes(data=True))
    shortest_paths = []  
    N = len(nodes)
    for i in range(N):
        paths_lists = self.bellman_ford_shortest_path(nodes[i][0]) 
        for p in paths_lists:
            shortest_paths.append(p)
    BC = {}
    for target_node in range(N):
        sum_ous = 0
        for path in shortest_paths:
            if path[0] != nodes[target_node][0] 
            and path[-1] != nodes[target_node][0] 
            and nodes[target_node][0] in path:
                sum_ous += 1
        BC[nodes[target_node][0]] = (sum_ous)/((N-1)*(N-2))
    return BC
```

In [62]:
%timeit P.betweenness_centrality()
%timeit R.betweenness_centrality()

1 loop, best of 5: 465 ms per loop
1 loop, best of 5: 11.4 s per loop


# Implementation decisions making
* In our implementation we used the Bellman-Ford algorithm (which is not good for this kind of problem) to calculate the shortest path between two nodes. This forced us to calculate the shortes path in both directions, therefore we needed to divide by 2 the result of the betweenness of the considered node. 

The total complexity of the this algorithm is $O(|V|^2|E|)$ while Floyd–Warshall algorithm takes $Θ(|V|^3)$

* Due to the great precision of the distances between the various nodes (graph **P** and **R**) and for the definition of the Bellman-Ford's algorithm itself, we thought it was appropriate to simplify the algorithm assuming that there could be at most one shortest path between each pair of nodes.

* Due to the large number of nodes we normalized the value of the betweenness so that $g(v)\in[0,1]$. The division is done by $\frac{(N−1)(N−2)}{2}$ cause we are working with an undirected graph.

* Based on the considerations made, the final formula for calculating the betweenness centrality of a node $v$ is

**$g(v) =\frac{\sum\limits_{s\neq v\neq t \in V} \frac{\sigma_{st}(v)}{\sigma_{st}}}{2}\frac{2}{(N-1)(N-2)} = \frac{\sum\limits_{s\neq v\neq t \in V} \frac{\sigma_{st}(v)}{\sigma_{st}}}{(N-1)(N-2)}$**

where $\sigma_{st}(v)$, $\sigma_{st} \in \{0,1\}$.
